## SianStock DCF

$$ DCF = \frac{FCFF_1}{(1 + r) ^ 1} + \frac{FCFF_2}{(1 + r) ^ 2} + ... + \frac{FCFF_n}{(1 + r) ^ n} $$

Where $FCFF$ stands for unlevered free cash flow for a given year ($FCFF_1$ means free cash flow in year 1, and so on); and $r$ stands for the discount rate.

$ FCFF = EBITDA - CAPEX - \Delta NWC - tax\_expenses $

Where:
1. EBITDA = earnings before interest, depreciation, and amortization, or pretax income,
2. CAPEX = capital expenditure, or fixed assets,
3. $ \Delta $ NWC = changes in net working capital, which can be derived by subtracting total current assets and total current liabilities in current and previous years,
4. tax_expenses is already self-explanatory (hopefully).

In [77]:
import os
import pandas
import requests
import string
from bs4 import BeautifulSoup
from functools import partial, reduce

In [78]:
INPUT_PATH = os.getcwd() + '/input/'
OUTPUT_PATH = os.getcwd() + '/output/'

In [79]:
def get_single_dataframe(year):
    """
    Returns dataframe in a given year.
    
    Argument:
        - year = four digits integer to get YYYY format.
    """
    
    # find files in the given directory in .csv format and store them in a list
    file_name = [x for x in os.listdir('input/') if x.endswith('{}.csv'.format(year))]
    
    # read each file in that list and store them in a list
    dfs = []
    for f in file_name:
        dfs.append(pandas.read_csv(INPUT_PATH + f))
        
    # merge them into a single dataframe
    return reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

In [80]:
def get_multiple_dataframes(year, subtrahend):
    """
    Returns multiple periods dataframe into a single dataframe.
    
    Arguments:
        - latest_year = four digits integer to get YYYY format.
        - subtrahend = a quantity or number to be subtracted from another.
    """
    
    # if subtrahend is 1 then total periods will be 2 years
    # example: 2018 - 1 = 2017, so it will be between 2017 and 2018
    # year + 1 is needed because otherwise it only ranges from 2017 to 2017
    dfs = []
    for y in range(year - subtrahend, year + 1):
        dfs.append(get_single_dataframe(year=y))
        
    return pandas.concat(dfs, sort = False).reset_index(drop = True)

In [81]:
baseline_dataframe = get_multiple_dataframes(2020, 1)

In [82]:
baseline_dataframe = get_multiple_dataframes(2020, 1)

## Calculate Free Cash Flow

In [83]:
def get_free_cash_flow(year):
    """
    Returns free cash flow dataframe.
    
    Argument:
        - year = four digits integer to get YYYY format.
    """
    
    # select necessary variables
    # sort by ticker code in ascending order
    # reset index
    nwc = baseline_dataframe[[
        'year', 
        'ticker_code', 
        'total_current_assets', 
        'total_current_liabilities'
    ]].sort_values(by=['ticker_code'], ascending=True) \
    .reset_index(drop=True)
    
    # add net working capital as a column
    nwc['net_working_capital'] = nwc['total_current_assets'] - nwc['total_current_liabilities']
    
    # changes in net working capital
    nwc['net_working_capital_delta'] = nwc.groupby('ticker_code')['net_working_capital'].diff()
    
    # filter dataframe only in year latest year
    nwc = nwc[nwc['year'] == year][[
        'ticker_code', 
        'net_working_capital_delta'
    ]].reset_index(drop=True)
    
    # calculate free cash flow
    fcff = baseline_dataframe[baseline_dataframe['year'] == year][[
        'year', 
        'ticker_code', 
        'pretax_income', 
        'fixed_assets', 
        'tax_expenses'
    ]]
    
    fcff = pandas.merge(fcff, nwc, how = 'inner', on = 'ticker_code').reset_index(drop = True)
    
    fcff['free_cash_flow'] = fcff['pretax_income'] \
    - fcff['fixed_assets'] \
    - fcff['net_working_capital_delta'] \
    - fcff['tax_expenses']
    
    return fcff[['year', 'ticker_code', 'free_cash_flow']].reset_index(drop=True)

In [84]:
fcff = get_free_cash_flow(2020)

## Calculate Discount Rate (r)

What we need to find out:
1. Cost of Equity 
2. Cost of Debt
3. Weighted Average Cost of Capital

### Cost of Equity

Cost of equity, or RE, can obtained by calculating [Capital Asset Pricing Model](https://www.investopedia.com/terms/c/capm.asp). Following are the items:

1. Risk-free rate
2. Beta
3. Equity market risk premium (market rate subtracted by risk-free rate)

After getting the items, we need to calculate the cost of equity as follows:

$$ RE = risk\_free\_rate + beta * (market\_rate - risk\_free\_rate) $$

### Risk-free rate

Risk-free rate is commonly derived from T-bills rate. In Indonesia, the equivalent is [Obligasi Negara](https://www.bi.go.id/id/moneter/obligasi-negara/Default.aspx), which is traded in the secondary market. Currently there are many traded bonds with various coupon rates, so I am going to use the average rate. First, we need to scrape the table to get the coupon rate.

### Cost of Debt

Cost of debt, or symbolized as RD, can be calculated as follows:

$$ RD = 1 - tax\_rate $$

According to [Trading Economics](https://tradingeconomics.com/indonesia/corporate-tax-rate), Indonesia's corporate tax rate is 25%. Hence, the cost of debt is 75%.

### Capital Structure (Weighted Average Cost of Capital)

In [85]:
risk_free_rate = 3.1

In [86]:
def get_cost_of_equity(beta, market_rate):
    """
    Returns capita asset pricing model for calculating cost of equity.
    
    Arguments:
        - risk_free_rate = see get_risk_free_rate function,
        - beta =  the covariance of asset returns with the market relative to variance of the market,
        - market_rate = expected stock returns of a given equity.
    """
    
    return risk_free_rate + beta * (market_rate - risk_free_rate)

In [87]:
cost_of_equity = get_cost_of_equity(0.05, 0.05)

In [88]:
def get_wacc(year, tax_rate=0.25):
    """
    Returns weighted average cost of capital.
    
    Arguments:
        - year = four digits integer to get YYYY format.
        - tax_rate = corporate tax rate in Indonesia. Basic value is 25%. Perhaps it is subject to change.
    """

    # get capital structure
    wacc = baseline_dataframe[baseline_dataframe['year'] == year][[
        'ticker_code', 
        'total_assets', 
        'total_equity'
    ]].reset_index(drop = True)
    
    wacc['equity_proportion'] = wacc['total_equity'] / wacc['total_assets']
    wacc['liabilities_proportion'] = 1 - wacc['equity_proportion']
    
    # cost of equity
    re = cost_of_equity
    
    # cost of debt
    rd = 1 - tax_rate
    
    # weighted cost of capital
    wacc['wacc'] = (re * wacc['equity_proportion']) + (rd * (1 - wacc['equity_proportion']))
    wacc = wacc[['ticker_code', 'wacc']]
    
    return wacc

In [89]:
wacc = get_wacc(2020, 0.05)

In [90]:
fcff = pandas.merge(fcff, wacc, how = 'inner', on = 'ticker_code')

## Calculate Fair Value

What we need to find out is the terminal value

$$ terminal\_value = \frac{FPYCF * (1 + LTCFGR)}{r - LTCFGR} $$



1. FPYCF = Final projected year cash flow
2. LTCGR = Long-term cash flow growth rate
3. r = discounted rate

In [91]:
def get_fair_value(growth_rate, total_period, year):
    
    """
    Returns fair value.
    
    Arguments:
        - growth_rate = projected cash flow growth rate,
        - num_year = total projected years,
        - year = four digits integer to get YYYY format.
    """
    
    fcff['fpycf'] = fcff['free_cash_flow'] * (1 + (growth_rate * total_period))
    fcff['terminal_value'] = (fcff['fpycf'] * (1 + growth_rate)) / (fcff['wacc'] - growth_rate)
    fcff['enterprise_value'] = fcff['free_cash_flow'] / ((1 + growth_rate) ** total_period)
    fcff['fair_value'] = fcff['terminal_value'] - fcff['enterprise_value']
    
    return fcff[['year', 'ticker_code', 'fair_value']]

In [92]:
final_dataframe = get_fair_value(0.0001, 1, 2020)

## Save Data

In [93]:
final_dataframe\
.sort_values(by='fair_value', ascending=False)\
.to_csv(OUTPUT_PATH + 'discounted-cash-flow-result.csv', index=False)